# Imports

In [8]:
from docarray import DocumentArray
from docarray import Document
from transformers import AutoModel, AutoTokenizer


/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-02 20:05:15.209502: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-02 20:05:15.209529: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:209: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
/usr/local/lib/python3.9/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:326: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Depre

# Setup

Import some documents:

In [7]:
da = DocumentArray(storage='weaviate', config={'host': 'weaviate'})

with da:
    da.extend(
        [
            Document(text='Persist Documents with Weaviate.'),
            Document(text='And enjoy fast nearest neighbor search.'),
            Document(text='All while using DocArray API.'),
        ]
    )


Create embeddings for the imported documents:

In [9]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')


def collate_fn(da):
    return tokenizer(da.texts, return_tensors='pt', truncation=True, padding=True)


da.embed(model, collate_fn=collate_fn)


Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 15.7kB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 445kB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 27.0MB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 30.2MB/s]
Downloading: 100%|██████████| 440M/440M [00:14<00:00, 30.8MB/s] 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from 

╭────────────────── Documents Summary ───────────────────╮
│                                                        │
│   Type                   DocumentArrayWeaviate         │
│   Length                 3                             │
│   Homogenous Documents   True                          │
│   Common Attributes      ('id', 'text', 'embedding')   │
│   Multimodal dataclass   False                         │
│                                                        │
╰────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ──────────────────────╮
│                                                                │
│   Attribute   Data type     #Unique values   Has empty value   │
│  ────────────────────────────────────────────────────────────  │
│   embedding   ('Tensor',)   3                False             │
│   id          ('str',)      3                False             │
│   text        ('str',)      3                False             │
│                                                                │
╰────────────────────────────────────────────────────────────────╯
╭──────────────────── DocumentArrayWeaviate Config ────────────────────╮
│                                                                      │
│   host                       weaviate                                │
│   port                       8080                                    │
│   protocol                   http                                    │
│   name                       Class6ba22a2830c84109bbcf4d88a30d9fdc   │
│   list_like                  True                                    │
│   serialize_config           {}                                      │
│   n_dim                      None                                    │
│   ef                         None                                    │
│   ef_construction            None                                    │
│   timeout_config             (10, 60)                                │
│   max_connections            None                                    │
│   dynamic_ef_min             None                                    │
│   dynamic_ef_max             None                                    │
│   dynamic_ef_factor          None                                    │
│   vector_cache_max_objects   None                                    │
│   flat_search_cutoff         None                                    │
│   cleanup_interval_seconds   None                                    │
│   skip                       None                                    │
│   columns                    {}                                      │
│   distance                   None                                    │
│   batch_size                 50                                      │
│   dynamic_batching           False                                   │
│   root_id                    True                                    │
│                                                                      │
╰──────────────────────────────────────────────────────────────────────╯

# Semantic Search

In [11]:
results = da.find(
    DocumentArray([Document(text='How to persist Documents')]).embed(
        model, collate_fn=collate_fn
    ),
    limit=1,
)

results[0].texts


['Persist Documents with Weaviate.']